In [1]:
import nltk
import os
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

In [2]:
class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words, labels):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, label_ids):
        self.input_ids = input_ids
        self.label_ids = label_ids

def read_examples_from_file(data_dir, mode):
    file_path = os.path.join(data_dir, "{}.txt".format(mode))
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(guid="{}-{}".format(mode, guid_index),
                                                 words=words,
                                                 labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    lab = splits[-3].replace("\n", "")
                    #print('#####################:',lab,splits[-4].replace("\n", ""))
                    if '$' in lab and len(list(lab))>1:
                        labels.append(lab[:])
                    elif not lab.isalnum():
                        labels.append("PUNC")
                    else:
                        labels.append(lab[:])
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("PUNC")
            #print(labels)
        if words:
            examples.append(InputExample(guid="%s-%d".format(mode, guid_index),
                                         words=words,
                                        labels=labels))
    return examples

In [3]:
def punc_check(i):
    labels =[]
    if '$' in i and len(list(i))>1:
        labels.append(i[:])
    elif not i.isalnum():
        labels.append("PUNC")
    else:
        labels.append(i[:])
    return((labels))

def nltk_evaluation():
    val_obj = read_examples_from_file('/home/shubham/Project/pos_tag/data/ner','dev')
    validation_data =[]
    for i in val_obj:
        validation_data.append((i.words, i.labels,))
    out_list = []
    pred_list = []
    for sentence, tags in (validation_data[:]): 
        pred_list.append([punc_check(i[1])[0] for i in nltk.pos_tag(sentence)])
        out_list.append(tags)
    sc = f1_score(out_list,pred_list)
    print(classification_report(out_list,pred_list))
    print(sc)
    return sc

In [4]:
nltk_evaluation()

           precision    recall  f1-score   support

      NNP       0.76      0.84      0.80      5817
       CD       0.87      0.86      0.87      2935
       IN       0.97      0.97      0.97      4892
      VBG       0.86      0.87      0.87       699
      VBD       0.92      0.94      0.93      2224
       NN       0.80      0.84      0.82      5358
       DT       0.99      0.99      0.99      3511
      VBZ       0.86      0.86      0.86       509
       VB       0.88      0.80      0.84      1112
       JJ       0.68      0.74      0.71      2765
      JJR       0.79      0.80      0.80       105
       CC       0.99      0.99      0.99       932
      VBN       0.87      0.80      0.83       928
     PUNC       0.93      0.89      0.91      6029
     PRP$       0.99      1.00      0.99       422
       RP       0.70      0.72      0.71       148
       RB       0.88      0.84      0.86       939
      VBP       0.75      0.84      0.79       365
      PRP       1.00      0.98

0.8793780275776099

In [5]:
"""lab = ['PRP', '$','VBD', 'EX', 'VBD', 'RB', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'NN', 'NN', 'IN', 'PRP$', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'JJ', 'NNS', '.']
def punc_check(i):
    labels =[]
    if '$' in i and len(list(i))>1:
        labels.append(i[:])
    elif not i.isalnum():
        labels.append("PUNC")
    else:
        labels.append(i[:])
    return((labels))"""

'lab = [\'PRP\', \'$\',\'VBD\', \'EX\', \'VBD\', \'RB\', \'NN\', \'IN\', \'JJ\', \'NNS\', \'IN\', \'NN\', \'NN\', \'IN\', \'PRP$\', \'JJ\', \'NN\', \'IN\', \'DT\', \'JJ\', \'JJ\', \'NNS\', \'.\']\ndef punc_check(i):\n    labels =[]\n    if \'$\' in i and len(list(i))>1:\n        labels.append(i[:])\n    elif not i.isalnum():\n        labels.append("PUNC")\n    else:\n        labels.append(i[:])\n    return((labels))'

In [6]:
"""print(type(punc_check('$')[0]))"""

"print(type(punc_check('$')[0]))"

# LSTM

In [7]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
import numpy as np

I0318 17:34:37.702981 140533864900416 file_utils.py:35] PyTorch version 1.2.0 available.
I0318 17:34:49.963968 140533864900416 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/shubham/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [8]:
EMBEDDING_DIM = 256
HIDDEN_DIM = 512
learning_rate = 5e-5
adam_epsilon = 1e-8
weight_decay = 0.0
lab_list = ['NNS', 'CD', 'TO', 'VBD', 'WP$', 'LS', 'RP', 'SYM', 'VBN', 'NNPS', 'RBR', 'JJS', 'VBP', 'MD', 'JJ', 'CC', 'VBG', 'IN', 'WP', 'PRP', 'PUNC', 'POS', 'FW', 'JJR', 'EX', 'WRB', 'DT', 'UH', 'VB', 'VBZ', 'RB', 'RBS', 'NN', 'WDT', 'NNP', 'PRP$', 'PDT']

label_map = {label:i for i, label in enumerate(lab_list)}
ix_to_tag = {i:label for i, label in enumerate(lab_list)}

class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores,tag_space

def label_selector(self,predicted_label_list,tokenized_sent):
        predicted_list =[]
        start = 0 
        end = 1 
        #print('check',tokenized_sent)
        print(len(predicted_label_list))
        for token in tokenized_sent:
            #print('check2',token)
            #print(start,end)
            window_len = len(tokenizer.tokenize(token))
            #print()
            #print((window_len))
            if window_len ==1:
                predicted_list.append(predicted_label_list[start])
                start=end
                end+=1
            elif window_len >1:
                end = window_len +start
                if (predicted_label_list[start] == 'PUNC'):
                    lab_check = False
                    for lab in predicted_label_list[start:end+1]:
                        if (lab != 'PUNC'):
                            predicted_list.append(lab)
                            lab_check = True
                            break
                    if (lab_check==False):
                        predicted_list.append(predicted_label_list[start])
                    else:
                        pass
                else:       
                    predicted_list.append(predicted_label_list[start])
                start=end
                end+=1
        return predicted_list

In [9]:
device = torch.device("cuda")
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, tokenizer.vocab_size, len(label_map))
model.load_state_dict(torch.load("/home/shubham/Project/pos_tag/code/distilation_experiments/lstm_models/model_0.8888022694298274.pt"))
model.to(device)
model.eval()

LSTMTagger(
  (word_embeddings): Embedding(30522, 256)
  (lstm): LSTM(256, 512, bidirectional=True)
  (hidden2tag): Linear(in_features=1024, out_features=37, bias=True)
)

In [45]:
txt = "let me google that for you"
txt  = '部分の装甲を強化するかを決定するために、被弾しながらも帰還した機体を調査しました。しかし撃墜された機体の被弾箇所については考慮しなかったため、最も攻撃を受けやすい機体部位を見落としました。 「生存者バイアス」とも呼ばれるこの現象は、成功を収めた結果のみを詳しく調査し、それに付随する失敗を見過ごしがちな人間の性質を表しています。私たちはあらゆるデータを検討しないままに、意見を採用したり、ビジネスを構築したり、意思決定を行ったりして、失敗に陥りやすいのです。 生存バイアスの詳細については、以下のTED Talkをご覧ください。 それでは、ビジネスにおける生存バイアスを回避するにはどうすればよいでしょうか？ 以下に、生存バイアスとしてよくある9つの例と、それぞれの生存バイアスの防止策をご紹介します。 この例はビジネスに直接関係するものではありませんが、生存バイアスという考え方の起源と見なされることが多いため、最初にご紹介しましょう。 第二次世界大戦中、連合軍は戦闘機の装甲を強化する必要があると考えました。しかし、材料に限りがあるため、機体全体に追加装甲を施すことはできません。そこで、専門家は最も攻撃を受けやすく、装甲を強化することで最大の効果が得られる部位を特定しなければなりませんでした。 追加装甲の部位を決定するうえで、専門家は被弾しながらも無事に帰還した機体を調査しました。調査の結果、帰還した機体のエンジンやコックピットには弾痕がなかったため、必然的にエンジンとコックピット以外のすべての部位の装甲を強化しようという結論に至りました。 幸いにも、数学者のAbraham Wald氏がこの計画の欠陥を指摘しました。無事に帰還した機体しか分析していないことが問題だと主張したのです。つまり被弾した箇所が機体にとって致命的ではなかったからこそ、その戦闘機は撃墜に至らずに済んだわけです。Wald氏は、帰還した機体の弾痕がなかった部位に装甲を追加することを軍に提案しました。 撃墜された原因を分析することで、Wald氏は生存バイアスという考え方を生み出し、このことは多くの操縦士の命を救うことにつながったと考えられます。 Googleで「大学を中退して成功した創業者」と検索してみると、世界屈指の有名企業がいくつかヒットします。ジョブズ氏、ゲイツ氏、ザッカーバーグ氏はいずれも優れたアイデアを胸に躍進し、奇跡的に成功を収めた起業家です。 しかし「多大な努力＝必ず成功」という先入観を持ってしまうと、重大な事実を見落とすことになります。大学中退者全体のうち、不運に見舞われている中退者の人数は決して少なくないはずです。 尊敬を集める創業者となりえたのは確かに懸命な努力があってこそですが、特定の状況でしか起こらなかった数々の出来事によって成功への道が開かれたとも考えられます。実際に、ある調査によると、米国で並外れた成功を収めた実業家の大部分（具体的には94％）は大学を卒業しています。 成功を収めるうえで学位は必要ないという仮説は、生存バイアスの一例です。もちろん例外はありますが、入手可能なすべての事実を検討してから結論を下すことが重要です。 「成功する人々の毎朝の習慣」、「すべての億万長者に共通する6つの特徴」、「ジェフ・ベゾスが語る成功に必要なたった1つの要因」、こういった記事をついつい読んでしまったという経験はないでしょうか？ かく言う私も、恥ずかしながら、似たような記事に幾度となく飛びついてきました。 私たちは、崇拝する人物について学べば同じように成功できると信じてやまないのです。ただし問題は、こういった記事でも、詳細な伝記でさえも、成功の追体験に必要なすべてのデータが提示されているわけではないという点です。一般読者は、背景にある変動要因を見落としたまま、不十分なデータを基に意思決定を行ってしまいます。 ベゾス氏がAmazonを立ち上げる以前から、同じような事業に挑戦して失敗した人はおそらく何百人もいるでしょう。しかし、そういった人に関する書籍はなかなか見つかりません。成功を収めた人物の成功を再現しようとしても、同様の結果が得られるとは限らないということを肝に銘じておきましょう。成功は努力以外に、周囲の環境や要因に大きく左右される場合もあるのです。 「次なるNetflix」、「○○業界のUber」。話題のスタートアップ企業に対してこんな表現が使用されているのを目にしたことがあるでしょう。しかし、米国中小企業庁によると、2年間存続した企業は全体のわずか3分の2で、5年以上だと全体の半分、10年以上になると全体の3分の1になります。 勝算は低く、成功を収めた企業のビジネスモデルに倣うだけでは、同様の成功を収められるとは限りません。 この手の生存バイアスの影響を受けて、多くの創業者は現在の市場、顧客、成長段階に適していないビジネスモデルを採用しようとします。 新規事業を立ち上げる場合には、尊敬する企業からインスピレーションを得ることも大切ですが、自社が置かれている現在の市場を評価して、アイデアを成功させるためにどのように進化するべきかを理解することが重要です。 デジタル ビデオ レコーダー（DVR）のTiVoが現在も事業を継続していることをご存じですか？ このDVRは、いち早く市場に出回り、製品としては現在でも優れています。全盛期にはTiVoが動詞として使われるほど大人気で、バラエティー番組を録画したりして5年以上はTiVoを使用していたという人も少なくありません。 TiVoの例は、優れた製品であっても、使いやすさやブランドロイヤルティーの勝負が始まり、市場に製品があふれる段階になると淘汰されることをよく表しています。高品質の製品やサービスを開発したからといって、当然のように顧客からの支持を集められると考えてはいけません（その典型が、わずか3年前にSketchersが市場シェアでAdidasを上回った一件です）。 生存バイアスを回避するためには、直接目に見えない他の変動要因も考慮する必要があります。 「成功する人々の毎朝の習慣」といった記事と同じくらいよく目にするのが、ある1つの変革によって劇的な成果を挙げた企業の事例と、その具体的な方法について説明する記事です。 しかし、生存バイアスから教訓を得るなら、同じような変革を試みてそれほど成果が得られなかった企業についても考慮する必要があります。 営業チームXがこのEメールテンプレートを使用して成約率を35％向上できたのが事実だとします。しかし、それに加えて新しい営業幹部を招き入れたり、報酬制度を改定したり、何か月も滞っていた数件の大口取引が偶然にも成約に至ったり、といった事実があっても、記事では言及されていない可能性があります。 このテンプレートの使用が成約率35％向上の唯一の要因だと考え、自社でも同じような効果を期待すると、高い確率で失望に終わることになります。 既存の顧客基盤のみに基づいて年間経常収益（ARR）を試算するなら、実際の収益が想定を下回るという厳しい現実を覚悟しなければいけません。 事業計画の中心となるのは確かに既存の顧客基盤ですが、生存バイアスを回避するためには、前年同期の顧客解約率を調査して、ARRの計算でも考慮に入れる必要があります。 そうすれば、想定される収益をより正確に把握できるため、将来的に見込みが外れて失望したり想定外のキャッシュフロー不足に陥ったりする事態を回避できます。 解約率を下げるために、満足度の低い顧客からクレームが来たら真摯に対応しようとしても、その顧客は既に失ったも同然という可能性が高いのです。ある調査では、SaaSユーザーの約半数はアカウントを無効化する2週間以上前に解約の手続きを開始しているという点が指摘されています。 したがって、顧客から苦情メールが送られてこないからといって、顧客が完全に満足しているものと思い込むのは危険です。Net Promoter Score®（NPS®）などのツールを使用して、顧客満足度と成長の可能性を測定しましょう。 満足度の高い顧客の声を把握することで、不安の兆候を早期に発見し、実際に問題が発生した場合でも、顧客が解約を決める前に先回りして対応することができます。 必要以上に機能を追加してしまう「フィーチャークリープ」という現象は大半の創業者が一度は直面する課題であり、顧客から要望があったという理由だけで、事業領域を広げて製品やサービスの機能を開発または実装し始めた場合に起こります。 市場、競合他社、顧客のニーズを把握することは確かに重要ですが、自社の本来の事業目的を見失わないことも等しく重要です。 機能を追加したところで、自社の主力製品／サービスの成長の妨げになるのでは、仮に少しは新規の顧客を獲得できたとしても、時間と労力に見合うほどの成果とは言えません。 機能を追加するメリットにだけ目を向けるのではなく、生存バイアスを考慮して、機能を追加することの影響をあらゆる角度から吟味してください。 生存バイアスを考慮することで、これまでとは違った新たな観点から問題を考えることができます。これは、多くの起業家が得意とする領域でしょう。意思決定を行う前に、入手可能なあらゆるデータを検討しましょう。それこそがビジネスの成否を決める鍵になるかもしれません。.'
txt = "Why don't you follow up on that production issue with doctor last week?"
txt = 'And Cat let us make sure that we are able to do a very good demonstration to show our AI capabilities in the upcoming call tonight.'
txt = 'Go ahead and create a ethercast for your team today or simply summarize the zoom recording in a YouTube video for your team.'
txt = 'Playing with the dog'
txt = txt.lower()
input_ids = tokenizer.encode(txt,add_special_tokens=False)
input_ids = torch.tensor(input_ids,device=device)
with torch.no_grad():
    tag_scores,_ = model(input_ids)
pred = []
for i in tag_scores:
    pred.append(ix_to_tag[int(np.argmax(i.cpu().detach().numpy()))])
#pred_list.append(pred)
print(pred)

['VBG', 'IN', 'DT', 'NN']


In [46]:
nltk.pos_tag(nltk.word_tokenize(txt))

[('playing', 'VBG'), ('with', 'IN'), ('the', 'DT'), ('dog', 'NN')]

In [47]:
from distilBiLstm_pos_tagger import DistilBiLstmPosTagger

In [48]:
a = DistilBiLstmPosTagger()

In [49]:
t = a.get_sent_pos_tags(txt)
#print(a.get_sent_pos_tags('lets google that.'))

In [50]:
print(t)

[('playing', 'VBG'), ('with', 'IN'), ('the', 'DT'), ('dog', 'NN')]


In [51]:
import sys
sys.path.insert(1,'/home/shubham/Project/pos_tag/code/inference/distilbert_pos_tagger')

from distilbert_pos_tagger import DistilBertPosTagger as dbpt
dbtag = dbpt('/home/shubham/Project/pos_tag/models/distil/pytorch_model.bin')

In [17]:
print(dbtag.get_sent_pos_tags(txt))

[('go', 'VB'), ('ahead', 'RB'), ('and', 'CC'), ('create', 'VB'), ('a', 'DT'), ('ethercast', 'NN'), ('for', 'IN'), ('your', 'PRP$'), ('team', 'NN'), ('today', 'NN'), ('or', 'CC'), ('simply', 'RB'), ('summarize', 'VB'), ('the', 'DT'), ('zoom', 'NN'), ('recording', 'NN'), ('in', 'IN'), ('a', 'DT'), ('youtube', 'NNP'), ('video', 'NN'), ('for', 'IN'), ('your', 'PRP$'), ('team', 'NN'), ('.', 'PUNC')]


# entity test

In [18]:
import pickle
from collections import Counter
sent_dict_ether = pickle.load(open("/home/shubham/Project/pos_tag/code/inference/ether_sent_dict_v1.pkl","rb"))

In [19]:
test_list = ['Colin','Api','React','S3','Web','Karthik','Lambda','Slack','Earth','Ui','God','Franklin','Google',
 'Id','Svg','Lambdas','Anna','Ios','Epa','Jesse','Mary','Firefox','Link','Chime','Pasha','Deep','Janus',
 'Ics','Windows', 'Gpt', 'Cullen', 'Gordon', 'Trish', 'Gothic', 'Zume', 'Zoom', 'Apa', 'Hermes', 'Ether',
 'Eddie', 'Url', 'Http', 'Python', 'Csv', 'Mitten', 'Aws', 'Gypsy', 'Microsoft', 'Get', 'Jason', 'Apis',
 'Sdk', 'Intel', 'Admin', 'Workspace', 'Aps', 'Safari', 'Multiple', 'Jitsi', 'Dm', 'App', 'Nisha', 'Christian',
 'Shank', 'Android', 'Netflix', 'Degraff', 'Ap', 'Ebs', 'Chrome', 'Fm', 'Ffmpeg', 'Jana', 'Native', 'Xcode',
 'Lake', 'Andre', 'Mittens', 'Collins', 'Internet', 'Cdn', 'Syrian', 'Gateway', 'Delorean', 'Etl', 'Nathan',
 'Turkish', 'Stripe', 'Javascript', 'Shivam', 'Sugar', 'Go', 'Cyril', 'Lee', 'Context', 'Ucs', 'Kota', 'Shubham',
 'Uk', 'Russian', 'Nato', 'Donald', 'Sean', 'Dynamodb', 'Mike', 'Dixie', 'Shri', 'Github', 'Michigan', 'Indian',
 'Psi', 'Margaret', 'Emily', 'Marcus', 'Golang', 'Phi', 'Allah', 'Tim', 'Arjun', 'Uuids', 'Janet', 'Alex', 'Richard',
 'Iphone', 'Uuid', 'Uid', 'Siwash', 'Shiva', 'Pimm', 'Manuel', 'Nick', 'Frank', 'English', 'AI', 'Sri',
 'Mithun', 'Kubernetes', 'Sql', 'Html', 'Pulseaudio',
 'Aw', 'Dfp', 'Akamai', 'Hadoop', 'Dot', 'Sqs', 'King', 'Lonnie', 'Nax', 'Nats', 'David', 'Postgres', 'One',
 'Lisa', 'Webrtc', 'Neo4j', 'Io', 'Youtube', 'Hls', 'Interstitial', 'Iris', 'Npm', 'Japan', 'Newton', 'Skull',
 'Ecs', 'Ec2', 'U.S', 'Jesus', 'Shivan', 'Graham', 'Node', 'Justin', 'Sdks', 'Localhost', 'Screen', 'Gpd',
 'Method', 'Tunisia', 'Tcp', 'Gps', 'Qa', 'Stephen', 'Pi', 'Martin', 'Sheree', 'Spacey', 'Elb', 'China', 'Ethan',
 'Morgan', 'Ooh', 'Linux', 'Src', 'Js', 'Lp', 'Kate',
 'Chris',
 'India',
 'Jack',
 'Nottingham',
 'Gts',
 'Objective-c',
 'Nsp']

In [20]:
for i in test_list[:]:
    print()
    print(i)
    print()
    nltk_word_pred = []
    bert_word_pred = []
    distilbert_word_pred =[]
    for j in sent_dict_ether[i]:
        #print(j)
        distilbert_pred = dict(dbtag.get_sent_pos_tags( j))
        bert_pred = dict(a.get_sent_pos_tags( j))
        nltk_pred = dict(nltk.pos_tag(nltk.word_tokenize(j)))
        try:
            nltk_word_pred.append(nltk_pred[i.lower()])
            bert_word_pred.append(bert_pred[i.lower()])
            distilbert_word_pred.append(distilbert_pred[i.lower()])
            #print(bert_pred[i.lower()],nltk_pred[i.lower()])
        except KeyError:
            #print('skipping',j)
            #print(nltk.word_tokenize(j))
            pass
    nltk_count = dict(Counter(nltk_word_pred))
    bert_count = dict(Counter(bert_word_pred))
    distilbert_count = dict(Counter(distilbert_word_pred))
    print('Total number of Sentences for',i,' :',len(nltk_word_pred))
    print('NLTK Predictions: ',(nltk_count))
    print('Bilstm Predictions:',bert_count)
    print('Distilbert Predictions:',distilbert_count)


Colin

Total number of Sentences for Colin  : 23
NLTK Predictions:  {'VBZ': 1, 'NN': 17, 'VB': 2, 'NNS': 2, 'VBN': 1}
Bilstm Predictions: {'NNP': 23}
Distilbert Predictions: {'NNP': 23}

Api

Total number of Sentences for Api  : 59
NLTK Predictions:  {'NN': 46, 'IN': 2, 'JJ': 7, 'NNS': 1, 'VB': 2, 'VBZ': 1}
Bilstm Predictions: {'RB': 35, 'JJ': 18, 'NN': 6}
Distilbert Predictions: {'NNP': 34, 'NN': 25}

React

Total number of Sentences for React  : 12
NLTK Predictions:  {'VB': 3, 'NN': 6, 'JJ': 3}
Bilstm Predictions: {'NNP': 1, 'VB': 2, 'NN': 3, 'JJ': 6}
Distilbert Predictions: {'VB': 12}

S3

Total number of Sentences for S3  : 26
NLTK Predictions:  {'NN': 22, 'JJ': 1, 'VB': 3}
Bilstm Predictions: {'NNP': 18, 'NN': 3, 'JJ': 3, 'POS': 1, 'VB': 1}
Distilbert Predictions: {'NNP': 15, 'CD': 4, 'NN': 7}

Web

Total number of Sentences for Web  : 84
NLTK Predictions:  {'NN': 83, 'JJ': 1}
Bilstm Predictions: {'NN': 19, 'NNP': 59, 'JJ': 6}
Distilbert Predictions: {'NN': 66, 'NNP': 18}

Karthi

Total number of Sentences for Mitten  : 5
NLTK Predictions:  {'VBN': 1, 'JJ': 2, 'NN': 2}
Bilstm Predictions: {'NNP': 5}
Distilbert Predictions: {'NNP': 4, 'VB': 1}

Aws

Total number of Sentences for Aws  : 14
NLTK Predictions:  {'NN': 5, 'NNS': 6, 'VBZ': 1, 'JJ': 2}
Bilstm Predictions: {'NNS': 10, 'VBZ': 1, 'NNP': 2, 'NNPS': 1}
Distilbert Predictions: {'NNP': 14}

Gypsy

Total number of Sentences for Gypsy  : 23
NLTK Predictions:  {'NN': 22, 'VB': 1}
Bilstm Predictions: {'NN': 22, 'VB': 1}
Distilbert Predictions: {'NNP': 10, 'NN': 13}

Microsoft

Total number of Sentences for Microsoft  : 5
NLTK Predictions:  {'NN': 4, 'JJ': 1}
Bilstm Predictions: {'NNP': 5}
Distilbert Predictions: {'NNP': 5}

Get

Total number of Sentences for Get  : 6
NLTK Predictions:  {'NN': 4, 'VB': 2}
Bilstm Predictions: {'NN': 3, 'VB': 3}
Distilbert Predictions: {'VBP': 2, 'VB': 4}

Jason

Total number of Sentences for Jason  : 17
NLTK Predictions:  {'NN': 16, 'VBP': 1}
Bilstm Predictions: {'NNP': 17}
Distilbe

Total number of Sentences for Context  : 10
NLTK Predictions:  {'NN': 9, 'JJ': 1}
Bilstm Predictions: {'NNP': 1, 'NN': 9}
Distilbert Predictions: {'NN': 10}

Ucs

Total number of Sentences for Ucs  : 2
NLTK Predictions:  {'JJ': 2}
Bilstm Predictions: {'NNS': 2}
Distilbert Predictions: {'NNP': 2}

Kota

Total number of Sentences for Kota  : 2
NLTK Predictions:  {'JJ': 2}
Bilstm Predictions: {'NNP': 2}
Distilbert Predictions: {'NNP': 2}

Shubham

Total number of Sentences for Shubham  : 12
NLTK Predictions:  {'NN': 11, 'JJ': 1}
Bilstm Predictions: {'NN': 9, 'VBP': 1, 'NNP': 2}
Distilbert Predictions: {'NNP': 12}

Uk

Total number of Sentences for Uk  : 4
NLTK Predictions:  {'NN': 3, 'JJ': 1}
Bilstm Predictions: {'NNP': 4}
Distilbert Predictions: {'NNP': 4}

Russian

Total number of Sentences for Russian  : 4
NLTK Predictions:  {'JJ': 4}
Bilstm Predictions: {'JJ': 4}
Distilbert Predictions: {'JJ': 4}

Nato

Total number of Sentences for Nato  : 3
NLTK Predictions:  {'NN': 2, 'JJ': 1}
Bils

Total number of Sentences for Nax  : 2
NLTK Predictions:  {'NN': 1, 'JJ': 1}
Bilstm Predictions: {'NNP': 2}
Distilbert Predictions: {'NNP': 1, 'JJ': 1}

Nats

Total number of Sentences for Nats  : 15
NLTK Predictions:  {'NNS': 13, 'RB': 1, 'VBZ': 1}
Bilstm Predictions: {'NNS': 14, 'NNP': 1}
Distilbert Predictions: {'NNS': 4, 'NNP': 8, 'NNPS': 2, 'VBZ': 1}

David

Total number of Sentences for David  : 2
NLTK Predictions:  {'JJ': 2}
Bilstm Predictions: {'NNP': 2}
Distilbert Predictions: {'NNP': 2}

Postgres

Total number of Sentences for Postgres  : 6
NLTK Predictions:  {'NNS': 4, 'VBZ': 2}
Bilstm Predictions: {'NN': 5, 'NNP': 1}
Distilbert Predictions: {'NNS': 5, 'NNP': 1}

One

Total number of Sentences for One  : 3
NLTK Predictions:  {'NN': 1, 'CD': 2}
Bilstm Predictions: {'NN': 1, 'CD': 2}
Distilbert Predictions: {'NN': 3}

Lisa

Total number of Sentences for Lisa  : 2
NLTK Predictions:  {'JJ': 1, 'NN': 1}
Bilstm Predictions: {'NNP': 2}
Distilbert Predictions: {'NNP': 2}

Webrtc

To

Total number of Sentences for Objective-c  : 3
NLTK Predictions:  {'JJ': 2, 'NN': 1}
Bilstm Predictions: {'NN': 3}
Distilbert Predictions: {'JJ': 2, 'NNP': 1}

Nsp

Total number of Sentences for Nsp  : 2
NLTK Predictions:  {'JJ': 2}
Bilstm Predictions: {'NNP': 2}
Distilbert Predictions: {'NNP': 2}
